In [2]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mkl-fft to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of mkl-random to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of mkl-umath to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.8 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 37.7 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
# 1. Install compatible NumPy and Scipy versions
!pip install "numpy<2.0" "scipy>=1.10" --upgrade --quiet

# 2. INSTRUCTION:
# After this cell finishes running, you MUST restart the Kernel.
# Go to the toolbar: Run > Restart Kernel (or click the 'Refresh' arrow button next to the Stop button).
# Then run the Training Cell below.
print("✅ Installation complete. PLEASE RESTART THE KERNEL NOW before running the next cell.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 69.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 MB 47.9 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
datasets 4.4.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
ydata-profiling 4.17.0 requires scipy<1.16,>=1.4.1, but you have scipy 1.16.3 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
google-colab 1.0.0 requires notebook==6.5.7, but you have notebook 6.5.4 which is incompatible.
google-cola

In [4]:
import os
import shutil
import yaml
import torch
import glob
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
# Load a model
model = YOLO("yolo11m.pt")  # load a pretrained model (recommended for training)

In [10]:
    import torch
    torch.cuda.empty_cache()

In [12]:
import torch
import gc

# 1. Delete the model variable
# This releases the model weights from the Python memory space.
if 'model' in locals():
    del model
    print("✅ Model object deleted.")

# 2. Clear the PyTorch CUDA cache
# This tells PyTorch to release unused memory chunks back to the GPU.
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("✅ PyTorch CUDA cache cleared.")

# 3. Run the Python Garbage Collector
gc.collect()
print("✅ Python garbage collector run.")

✅ Model object deleted.
✅ PyTorch CUDA cache cleared.
✅ Python garbage collector run.


In [8]:
# STOP the current training run immediately.
# Then, execute these commands in a new cell before restarting training:
import shutil
import os

# Path to your corrupted run directory
CORRUPTED_RUN_PATH = f"/kaggle/working/runs/detect/"

if os.path.exists(CORRUPTED_RUN_PATH):
    shutil.rmtree(CORRUPTED_RUN_PATH)
    print(f"🧹 Successfully cleaned up corrupted run directory: {CORRUPTED_RUN_PATH}")
else:
    print("Directory not found, ready to start new training.")

🧹 Successfully cleaned up corrupted run directory: /kaggle/working/runs/detect/


In [ ]:
import os
import shutil
import yaml
from ultralytics import YOLO
import torch

# ==========================================
# 1. CONFIGURATION
# ==========================================
PROJECT_NAME = "Forest_Fire_Smoke_Detection"
RUN_NAME = "yolov11_fire_smoke_v1_STABLE" 

# INPUT: Your curated dataset
INPUT_DATASET_DIR = "/kaggle/input/fire-smoke-v1/dataset_split-V1" 

# HYPERPARAMETERS (Tuned for P100 16GB & Night/Cloud Robustness)
IMGSZ = 640          # 640 is standard/stable.
BATCH_SIZE = 16      # Increased for better Batch Normalization stability.
EPOCHS = 50

# ==========================================
# 2. HIGH-SPEED DATA SETUP
# ==========================================
print("🚀 Setting up Industrial Training Environment...")

ROOT_WORKING_DIR = "/kaggle/working/"
DATASET_NAME = INPUT_DATASET_DIR.split('/')[-1] 
DATA_DIR = os.path.join(ROOT_WORKING_DIR, DATASET_NAME)

def setup_data(source_dir, target_dir):
    if not os.path.exists(source_dir):
        raise FileNotFoundError(f"❌ Input dataset not found at {source_dir}.")
    
    if os.path.exists(target_dir):
        print(f"ℹ️ Data already exists in {target_dir}. Skipping copy.")
    else:
        print(f"📂 Copying data to local drive for High-Speed I/O... (Please wait)")
        shutil.copytree(source_dir, target_dir)
        print("✅ Data copy complete.")
    return target_dir

# Execute Setup
setup_data(INPUT_DATASET_DIR, DATA_DIR)

# Generate data.yaml
yaml_content = {
    'path': DATA_DIR, 
    'train': 'train/images',  
    'val': 'valid/images',    
    'test': 'test/images',
    'names': {0: 'smoke', 1: 'fire'} 
}

yaml_path = '/kaggle/working/data.yaml'
with open(yaml_path, 'w') as f:
    yaml.dump(yaml_content, f)
print(f"✅ Config saved to {yaml_path}")

# --- Everything below Section 3 has been updated for stability ---
# ==========================================
# 3. ROBUST TRAINING LOOP (FORCED STABILITY)
# ==========================================

# Check for resume
last_ckpt = f"/kaggle/working/runs/detect/{RUN_NAME}/weights/last.pt"
resume_training = os.path.exists(last_ckpt)

if resume_training:
    print(f"♻️ RESUMING training from {last_ckpt}")
    model = YOLO(last_ckpt)
else:
    print("🆕 Starting NEW training session")
    model = YOLO('yolo11m.pt') 

print("🔥 Initiating Training with FORCED AdamW STABILITY MODE...")

try:
    results = model.train(
        data=yaml_path,
        project="/kaggle/working/runs/detect",
        name=RUN_NAME,
        epochs=EPOCHS,
        imgsz=IMGSZ,
        batch=BATCH_SIZE,
        
        # --- CRITICAL STABILITY FIXES (FORCED ADAMW) ---
        optimizer='AdamW',         # FORCED: AdamW is stable at low LR
        lr0=0.001,                 # FORCED: Safe, low initial Learning Rate
        cos_lr=True,               # Use Cosine LR scheduler for smooth decay
        warmup_epochs=3.0,
        
        # --- NIGHT MODE & ROBUSTNESS ---
        hsv_h=0.015,
        hsv_s=0.7,
        hsv_v=0.4,                 # High V for simulating low light/glare
        
        # --- CONTEXT & HARD NEGATIVES ---
        mosaic=1.0,                # Critical for learning small smoke/context
        mixup=0.1,
        degrees=10.0,              # For UAV robustness
        fliplr=0.5,
        close_mosaic=10,
        
        # --- SYSTEM ---
        device=0,
        workers=4,
        save=True,
        save_period=3,
        resume=resume_training,
        plots=True
    )
    print("🎉 Training Completed Successfully!")

except Exception as e:
    print(f"⚠️ Training Interrupted: {e}")
    print("To resume, simply re-run this cell.")

🚀 Setting up Industrial Training Environment...
ℹ️ Data already exists in /kaggle/working/dataset_split-V1. Skipping copy.
✅ Config saved to /kaggle/working/data.yaml
🆕 Starting NEW training session
🔥 Initiating Training with FORCED AdamW STABILITY MODE...
Ultralytics 8.3.233 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/kaggle/working/data.yaml, degrees=10.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=

In [1]:
import os
import shutil
import yaml
from ultralytics import YOLO
import torch

# ==========================================
# 1. CONFIGURATION
# ==========================================
PROJECT_NAME = "Forest_Fire_Smoke_Detection"
RUN_NAME = "yolov11_fire_smoke_v1_STABLE" 

# INPUT: Your curated dataset
INPUT_DATASET_DIR = "/kaggle/input/fire-smoke-v1/dataset_split-V1" 

# HYPERPARAMETERS (Tuned for P100 16GB & Night/Cloud Robustness)
IMGSZ = 640          # 640 is standard/stable.
BATCH_SIZE = 16      # Increased for better Batch Normalization stability.
EPOCHS = 50

# ==========================================
# 2. HIGH-SPEED DATA SETUP
# ==========================================
print("🚀 Setting up Industrial Training Environment...")

ROOT_WORKING_DIR = "/kaggle/working/"
DATASET_NAME = INPUT_DATASET_DIR.split('/')[-1] 
DATA_DIR = os.path.join(ROOT_WORKING_DIR, DATASET_NAME)

def setup_data(source_dir, target_dir):
    if not os.path.exists(source_dir):
        raise FileNotFoundError(f"❌ Input dataset not found at {source_dir}.")
    
    if os.path.exists(target_dir):
        print(f"ℹ️ Data already exists in {target_dir}. Skipping copy.")
    else:
        print(f"📂 Copying data to local drive for High-Speed I/O... (Please wait)")
        shutil.copytree(source_dir, target_dir)
        print("✅ Data copy complete.")
    return target_dir

# Execute Setup
setup_data(INPUT_DATASET_DIR, DATA_DIR)

# Generate data.yaml
yaml_content = {
    'path': DATA_DIR, 
    'train': 'train/images',  
    'val': 'valid/images',    
    'test': 'test/images',
    'names': {0: 'smoke', 1: 'fire'} 
}

yaml_path = '/kaggle/working/data.yaml'
with open(yaml_path, 'w') as f:
    yaml.dump(yaml_content, f)
print(f"✅ Config saved to {yaml_path}")

# --- Everything below Section 3 has been updated for stability ---
# ==========================================
# 3. ROBUST TRAINING LOOP (FORCED STABILITY)
# ==========================================

# Check for resume
last_ckpt = f"/kaggle/working/runs/detect/{RUN_NAME}/weights/last.pt"
resume_training = os.path.exists(last_ckpt)

if resume_training:
    print(f"♻️ RESUMING training from {last_ckpt}")
    model = YOLO(last_ckpt)
else:
    print("🆕 Starting NEW training session")
    model = YOLO('yolo11m.pt') 

print("🔥 Initiating Training with FORCED AdamW STABILITY MODE...")

try:
    results = model.train(
        data=yaml_path,
        project="/kaggle/working/runs/detect",
        name=RUN_NAME,
        epochs=EPOCHS,
        imgsz=IMGSZ,
        batch=BATCH_SIZE,
        
        # --- CRITICAL STABILITY FIXES (FORCED ADAMW) ---
        optimizer='AdamW',         # FORCED: AdamW is stable at low LR
        lr0=0.001,                 # FORCED: Safe, low initial Learning Rate
        cos_lr=True,               # Use Cosine LR scheduler for smooth decay
        warmup_epochs=3.0,
        
        # --- NIGHT MODE & ROBUSTNESS ---
        hsv_h=0.015,
        hsv_s=0.7,
        hsv_v=0.4,                 # High V for simulating low light/glare
        
        # --- CONTEXT & HARD NEGATIVES ---
        mosaic=1.0,                # Critical for learning small smoke/context
        mixup=0.1,
        degrees=10.0,              # For UAV robustness
        fliplr=0.5,
        close_mosaic=10,
        
        # --- SYSTEM ---
        device=0,
        workers=4,
        save=True,
        save_period=3,
        resume=resume_training,
        plots=True
    )
    print("🎉 Training Completed Successfully!")

except Exception as e:
    print(f"⚠️ Training Interrupted: {e}")
    print("To resume, simply re-run this cell.")

🚀 Setting up Industrial Training Environment...
ℹ️ Data already exists in /kaggle/working/dataset_split-V1. Skipping copy.
✅ Config saved to /kaggle/working/data.yaml
♻️ RESUMING training from /kaggle/working/runs/detect/yolov11_fire_smoke_v1_STABLE/weights/last.pt
🔥 Initiating Training with FORCED AdamW STABILITY MODE...
Ultralytics 8.3.233 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/kaggle/working/data.yaml, degrees=10.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=Fals

In [8]:
# ==========================================
# 4. VALIDATION ON TEST SET (ADDED CONF/IOU)
# ==========================================
# This validates specifically on your 'Test' split (which you mentioned creating)
print("\n📊 Running Final Validation on TEST set...")
try:
    # Load best weights
    best_model = YOLO(f"/kaggle/working/runs/detect/yolov11_fire_smoke_v1_STABLE/weights/best.pt")
    
    # We set a low confidence threshold (conf=0.01) to find ALL possible detections, 
    # ensuring the model isn't mistakenly detecting clouds/glare (your hard negatives).
    metrics = best_model.val(
        split='test', 
        save_json=True, 
        plots=True, 
        conf=0.01, # Low threshold to check for false positives
        iou=0.6    # Standard IoU
    )
    
    print(f"🔥 Map50: {metrics.box.map50}")
    print(f"🔥 Map50-95: {metrics.box.map}")
    
except Exception as e:
    print(f"⚠️ Validation skipped or failed: {e}")


📊 Running Final Validation on TEST set...
Ultralytics 8.3.234 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
YOLO11m summary (fused): 125 layers, 20,031,574 parameters, 0 gradients, 67.7 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 67.6±67.1 MB/s, size: 413.5 KB)
val: Scanning /kaggle/working/dataset_split-V1/test/labels... 2976 images, 1089 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 2976/2976 668.3it/s 4.5s0.1s
val: /kaggle/working/dataset_split-V1/test/images/WEB07534.jpg: corrupt JPEG restored and saved
val: /kaggle/working/dataset_split-V1/test/images/WEB11313.jpg: corrupt JPEG restored and saved
val: New cache created: /kaggle/working/dataset_split-V1/test/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 186/186 4.3it/s 43.1s0.3ss


/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all       2976       3345      0.791      0.735      0.809      0.506
                 smoke       1776       1928      0.869      0.804       0.88      0.603
                  fire        590       1417      0.713      0.665      0.739       0.41
Speed: 0.6ms preprocess, 8.3ms inference, 0.0ms loss, 0.6ms postprocess per image
Saving /kaggle/working/runs/detect/val/predictions.json...
Results saved to /kaggle/working/runs/detect/val
🔥 Map50: 0.8094746435567568
🔥 Map50-95: 0.5063029893996813


In [14]:
import torch

# Path to the file
ckpt_path = "/kaggle/working/runs/detect/yolov11_fire_smoke_v1_STABLE/weights/last.pt"

print(f"🔧 Fixing checkpoint state: {ckpt_path}")

try:
    # 1. Load with weights_only=False (Critical for your environment)
    checkpoint = torch.load(ckpt_path, map_location='cpu', weights_only=False)

    # 2. FORCE the internal counters
    # We set 'epoch' to 49. In 0-indexing, this means 50 epochs are done.
    # The next epoch will be index 50 (displayed as Epoch 51).
    checkpoint['epoch'] = 49  
    
    # Ensure the goal is 100
    if 'train_args' in checkpoint:
        checkpoint['train_args']['epochs'] = 100
    
    # 3. Save the fixed file
    torch.save(checkpoint, ckpt_path)
    print("✅ Checkpoint fixed: Set internal 'epoch' to 49 and 'goal' to 100.")

except Exception as e:
    print(f"❌ Error: {e}")

🔧 Fixing checkpoint state: /kaggle/working/runs/detect/yolov11_fire_smoke_v1_STABLE/weights/last.pt
✅ Checkpoint fixed: Set internal 'epoch' to 49 and 'goal' to 100.


In [ ]:
import os
import shutil
import yaml
from ultralytics import YOLO
import torch

# ==========================================
# 1. CONFIGURATION
# ==========================================
PROJECT_NAME = "Forest_Fire_Smoke_Detection"
RUN_NAME = "yolov11_fire_smoke_v1_STABLE" 

# INPUT: Your curated dataset
INPUT_DATASET_DIR = "/kaggle/input/fire-smoke-v1/dataset_split-V1" 

# HYPERPARAMETERS (Tuned for P100 16GB & Night/Cloud Robustness)
IMGSZ = 640          # 640 is standard/stable.
BATCH_SIZE = 16      # Increased for better Batch Normalization stability.
EPOCHS = 100

# ==========================================
# 2. HIGH-SPEED DATA SETUP
# ==========================================
print("🚀 Setting up Industrial Training Environment...")

ROOT_WORKING_DIR = "/kaggle/working/"
DATASET_NAME = INPUT_DATASET_DIR.split('/')[-1] 
DATA_DIR = os.path.join(ROOT_WORKING_DIR, DATASET_NAME)

def setup_data(source_dir, target_dir):
    if not os.path.exists(source_dir):
        raise FileNotFoundError(f"❌ Input dataset not found at {source_dir}.")
    
    if os.path.exists(target_dir):
        print(f"ℹ️ Data already exists in {target_dir}. Skipping copy.")
    else:
        print(f"📂 Copying data to local drive for High-Speed I/O... (Please wait)")
        shutil.copytree(source_dir, target_dir)
        print("✅ Data copy complete.")
    return target_dir

# Execute Setup
setup_data(INPUT_DATASET_DIR, DATA_DIR)

# Generate data.yaml
yaml_content = {
    'path': DATA_DIR, 
    'train': 'train/images',  
    'val': 'valid/images',    
    'test': 'test/images',
    'names': {0: 'smoke', 1: 'fire'} 
}

yaml_path = '/kaggle/working/data.yaml'
with open(yaml_path, 'w') as f:
    yaml.dump(yaml_content, f)
print(f"✅ Config saved to {yaml_path}")

# --- Everything below Section 3 has been updated for stability ---
# ==========================================
# 3. ROBUST TRAINING LOOP (FORCED STABILITY)
# ==========================================

# Check for resume
last_ckpt = "/kaggle/working/runs/detect/yolov11_fire_smoke_v1_STABLE/weights/last.pt"
resume_training = os.path.exists(last_ckpt)

if resume_training:
    print(f"♻️ RESUMING training from {last_ckpt}")
    model = YOLO(last_ckpt)
else:
    print("🆕 Starting NEW training session")
    model = YOLO('yolo11m.pt') 

print("🔥 Initiating Training with FORCED AdamW STABILITY MODE...")

try:
    results = model.train(
        data=yaml_path,
        project="/kaggle/working/runs/detect",
        name=RUN_NAME,
        epochs=EPOCHS,
        imgsz=IMGSZ,
        batch=BATCH_SIZE,
        
        # --- CRITICAL STABILITY FIXES (FORCED ADAMW) ---
        optimizer='AdamW',         # FORCED: AdamW is stable at low LR
        lr0=0.001,                 # FORCED: Safe, low initial Learning Rate
        cos_lr=True,               # Use Cosine LR scheduler for smooth decay
        warmup_epochs=3.0,
        
        # --- NIGHT MODE & ROBUSTNESS ---
        hsv_h=0.015,
        hsv_s=0.7,
        hsv_v=0.4,                 # High V for simulating low light/glare
        
        # --- CONTEXT & HARD NEGATIVES ---
        mosaic=1.0,                # Critical for learning small smoke/context
        mixup=0.1,
        degrees=10.0,              # For UAV robustness
        fliplr=0.5,
        close_mosaic=10,
        
        # --- SYSTEM ---
        device=0,
        workers=4,
        save=True,
        save_period=3,
        resume=resume_training,
        plots=True
    )
    print("🎉 Training Completed Successfully!")

except Exception as e:
    print(f"⚠️ Training Interrupted: {e}")
    print("To resume, simply re-run this cell.")

🚀 Setting up Industrial Training Environment...
ℹ️ Data already exists in /kaggle/working/dataset_split-V1. Skipping copy.
✅ Config saved to /kaggle/working/data.yaml
♻️ RESUMING training from /kaggle/working/runs/detect/yolov11_fire_smoke_v1_STABLE/weights/last.pt
🔥 Initiating Training with FORCED AdamW STABILITY MODE...
Ultralytics 8.3.234 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/kaggle/working/data.yaml, degrees=10.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=Fal

In [5]:
# ==========================================
# 4. VALIDATION ON TEST SET (ADDED CONF/IOU)
# ==========================================
# This validates specifically on your 'Test' split (which you mentioned creating)
print("\n📊 Running Final Validation on TEST set...")
try:
    # Load best weights
    best_model = YOLO(f"/kaggle/working/runs/detect/yolov11_fire_smoke_v1_STABLE8/weights/best.pt")
    
    # We set a low confidence threshold (conf=0.01) to find ALL possible detections, 
    # ensuring the model isn't mistakenly detecting clouds/glare (your hard negatives).
    metrics = best_model.val(
        split='test', 
        save_json=True, 
        plots=True, 
        conf=0.01, # Low threshold to check for false positives
        iou=0.6    # Standard IoU
    )
    
    print(f"🔥 Map50: {metrics.box.map50}")
    print(f"🔥 Map50-95: {metrics.box.map}")
    
except Exception as e:
    print(f"⚠️ Validation skipped or failed: {e}")


📊 Running Final Validation on TEST set...
Ultralytics 8.3.235 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon CPU @ 2.20GHz)
YOLO11m summary (fused): 125 layers, 20,031,574 parameters, 0 gradients, 67.7 GFLOPs
val: Fast image access ✅ (ping: 0.1±0.0 ms, read: 52.8±49.7 MB/s, size: 797.9 KB)
val: Scanning /kaggle/working/dataset_split-V1/test/labels.cache... 2976 images, 1089 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 2976/2976 2.4Mit/s 0.0s0s
val: /kaggle/working/dataset_split-V1/test/images/WEB07534.jpg: corrupt JPEG restored and saved
val: /kaggle/working/dataset_split-V1/test/images/WEB11313.jpg: corrupt JPEG restored and saved
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 186/186 10.8s/it 33:3712.9s


/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all       2976       3345      0.813      0.742      0.821      0.516
                 smoke       1776       1928      0.888      0.809       0.89      0.616
                  fire        590       1417      0.737      0.674      0.753      0.416
Speed: 1.9ms preprocess, 652.2ms inference, 0.0ms loss, 0.4ms postprocess per image
Saving /kaggle/working/runs/detect/val2/predictions.json...
Results saved to /kaggle/working/runs/detect/val2
🔥 Map50: 0.8214262521935696
🔥 Map50-95: 0.5160459036959522
